DATTABAYÖ - Borja Esteve Molner & Jorge López Fresco

## Sesión 1/5 - Práctica 3 - LNR
### Preprocesamiento y extracción de características

**Objetivo**: El objetivo de esta primera práctica es cargar los datos y preprocersarlos, realizar una extracción de características para finalmente realizar una representación.

**Referencias y Documentación**:
- Preprocesamiento: 
  - Análisis de Stopwords: https://www.nltk.org/
  - FreeLing: https://freeling-user-manual.readthedocs.io/en/latest/toc/
  - Spacy: https://spacy.io/usage/spacy-101
    
    
- Algoritmos para construir word embedding:
  - Word2Vec: https://arxiv.org/abs/1301.3781 y https://arxiv.org/abs/1310.4546
  - GloVe: https://nlp.stanford.edu/projects/glove/
  - FastText: https://fasttext.cc/
  - GEMSIM: https://pypi.org/project/gensim/ y https://github.com/RaRe-Technologies/gensim-data

**Importar Librerías**:

In [ ]:
# Instalar FastText
!pip install fasttext

In [ ]:
# Instalar Spacy
!pip install spacy
!python -m spacy download es_core_news_sm

In [ ]:
!wget http://dcc.uchile.cl/~jperez/word-embeddings/fasttext-sbwc.vec.gz

In [55]:
import spacy
import nltk
from nltk.corpus import stopwords
import gensim
import pandas as pd
import numpy as np
import es_core_news_sm
import gensim.downloader as api
from scipy.spatial import distance
from gensim.models import Word2Vec
from spacy.lang.es.examples import sentences
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.wrappers.fasttext import FastText as ft
from gensim.models.keyedvectors import KeyedVectors
# Cargar modelo de Spacy
nlp = es_core_news_sm.load()
nltk.download('stopwords')

In [8]:
from google.colab import drive
drive.mount('/content/drive')
# Añadir la ruta de Google Drive en donde está el archivo "train.csv"
ruta= "/content/drive/MyDrive/3º - GCD/2C - LENGUAJE NATURAL Y RECUPERACIÓN DE LA INFORMACIÓN/Prácticas LNR/Práctica 3 (Sesiones 1 a 5) - LNR/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
df_detests=pd.read_csv(ruta+"train.csv")
df_detests.head(20)[["sentence","stereotype"]]

,sentence,stereotype
0,La solución es desarrollar el pensamiento crít...,0
1,Hay que enseñar que la magia no existe.,0
2,Que todos los avances de la humanidad siempre ...,0
3,Enseñar en las escuelas la historia de las rel...,0
4,Desde las religiones de la edad de piedra hast...,0
5,Después de eso da igual lo que intenten adoctr...,0
6,"Pero no es posible prohibir las religiones, po...",0
7,¿Manipulando?.,0
8,"Un hecho real, que no me impide NO demonizar a...",0
9,No se donde ves la manipulación.,0


## Tokenizado y lematizado.

El primer paso que realizaremos será obtener una lista de palabras tokenizadas, con el texto original y su lema, teniendo en cuenta el tipo de elemento sintáctico al que se refiere. Nuestro objetivo es observar la estructura de las frases y su tipología. Así, podremos saber qué elementos mantener y cuáles eliminar u omitir.

In [104]:
frases = df_detests["sentence"]
tokens = [ (t.text, t.pos_, t.lemma_) for i,f in enumerate(frases) 
                                 for t in nlp(frases.to_list()[i])]

In [105]:
cont = 0
for t1,t2,t3 in tokens:
  if cont < 50:
    print(t1,t2,t3)
  cont += 1

La DET La
solución NOUN solución
es AUX ser
desarrollar VERB desarrollar
el DET el
pensamiento NOUN pensamiento
crítico ADJ crítico
y CONJ y
escéptico ADJ escéptico
. PUNCT .
Hay AUX Hay
que SCONJ que
enseñar VERB enseñar
que SCONJ que
la DET lo
magia NOUN magia
no ADV no
existe VERB existir
. PUNCT .
Que SCONJ Que
todos DET todo
los DET lo
avances NOUN avanzar
de ADP de
la DET lo
humanidad NOUN humanidad
siempre ADV siempre
han AUX haber
sido AUX ser
a ADP a
pesar NOUN pesar
de ADP de
las DET los
religiones NOUN religión
y CONJ y
nunca ADV nunca
gracias NOUN gracia
a ADP a
ellas PRON ellos
. PUNCT .
Enseñar VERB Enseñar
en ADP en
las DET los
escuelas NOUN escuela
la DET lo
historia NOUN historia
de ADP de
las DET los
religiones NOUN religión
, PUNCT ,


## Preprocesado

Para realizar el preprocesado, llevaremos a cabo varias acciones. Una de ellas consistirá en normalizar y convertir todas las palabras a minúsculas. Por otra parte, eliminaremos signos de puntuación, espacios y caracteres raros.  

En lo que respecta a las StopWords, haremos una pequeña prueba empírica para decidir si eliminarlas o no. Para ello utilizaremos las 50 primeras oraciones y obtendremos el valor medio de los vectores de Word Embedding usando Word2Vec. A continuación, realizaremos el mismo proceso, pero eliminando las StopWords. De esta forma, si se observan diferencias en las medias de las oraciones, eso significará que las Stopwords no afectan a la representación y por tanto, optaremos por eliminarlas para mejorar la capacidad de cómputo. 

Normalizar, convertir a minúsculas y eliminar signos de puntuación, espacios y caracteres raros.

In [107]:
words, sentences = [], []
for i,f in enumerate(frases):
    sents, orac = [], ''
    sentence = frases.to_list()[i]
    doc = nlp(sentence)
    for token in doc:
        if token.pos_ != 'SPACE' and token.pos_ != 'PUNCT':
            sents.append(token.text.lower())
            orac += token.text.lower() + ' '
    words.append(sents)
    sentences.append(orac.rstrip())

In [109]:
# Ejemplo de las palabras y de frases guardadas.
print(words[:2])
print(sentences[:2])

[['la', 'solución', 'es', 'desarrollar', 'el', 'pensamiento', 'crítico', 'y', 'escéptico'], ['hay', 'que', 'enseñar', 'que', 'la', 'magia', 'no', 'existe']]
['la solución es desarrollar el pensamiento crítico y escéptico', 'hay que enseñar que la magia no existe']


Comparativa sobre la representación con StopWords y sin ellas.

In [110]:
s_words = set(stopwords.words('spanish'))
words_SW, sentences_SW = [], []
for i,f in enumerate(frases):
    sents, orac = [], ''
    sentence = frases.to_list()[i]
    doc = nlp(sentence)
    for token in doc:
        if token.pos_ != 'SPACE' and token.pos_ != 'PUNCT' and token.text not in s_words:
            sents.append(token.text.lower())
            orac += token.text.lower() + ' '
    words_SW.append(sents)
    sentences_SW.append(orac.rstrip())

In [111]:
# Ejemplo de las palabras y de frases guardadas sin las StopWords.
print(words_SW[:2])
print(sentences_SW[:2])

[['la', 'solución', 'desarrollar', 'pensamiento', 'crítico', 'escéptico'], ['hay', 'enseñar', 'magia', 'existe']]
['la solución desarrollar pensamiento crítico escéptico', 'hay enseñar magia existe']


In [56]:
wordvectors_file_vec = 'fasttext-sbwc.vec.gz'
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [113]:
medias = []
for i,sent in enumerate(words):
  suma, cont = 0, 1
  for w in sent:
    try:
      y = np.mean(wordvectors.get_vector(w))
      suma += y
      cont += 1
    except KeyError:
      suma += 0
      cont += 1
  media = suma / cont
  medias.append(abs(round(media,4)))

medias_SW = []
for i,sent in enumerate(words_SW):
  suma, cont = 0, 1
  for w in sent:
    try:
      y = np.mean(wordvectors.get_vector(w))
      suma += y
      cont += 1
    except KeyError:
      suma += 0
      cont += 1
  media = suma / cont
  medias_SW.append(abs(round(media,4)))

In [114]:
# Ejemplo de los vectores de medias resultantes.
print(medias[:4],"...")
print(medias_SW[:4],"...")

[0.0039, 0.0021, 0.0023, 0.0027] ...
[0.0038, 0.0042, 0.0027, 0.0012] ...


In [116]:
print("Max y min de medias:" ,max(medias), min(medias))
print("Max y min de medias_SW:" ,max(medias_SW), min(medias_SW))
# Creamos un valor para considerar significativa la diferencia entre
# las medias de ambas muestras. Usaremos un valor que suponga un 5%
# del valor obtenido de restar el máximo menos el mínimo de las medias. 
epsilon = max(medias)*0.05
print("Epsilon:", epsilon)

Max y min de medias: 0.0184 0.0
Max y min de medias_SW: 0.0184 0.0
Epsilon: 0.00092


In [117]:
iguales, distintos = 0, 0
for m1,m2 in zip(medias,medias_SW):
    if abs(m1) - abs(m2) > epsilon:
      distintos += 1
    else:
      iguales += 1      

In [118]:
print("Número de valores distintos entre SW y sin SW:", distintos)
print("Número de valores iguales entre SW y sin SW:", iguales)
print("Porcentaje Distintos: ", 503 * 100 / len(medias), "%")
print("Porcentaje Iguales: ", 3314 * 100 / len(medias), "%")

Número de valores distintos entre SW y sin SW: 503
Número de valores iguales entre SW y sin SW: 3314
Porcentaje Distintos:  13.177888394026722 %
Porcentaje Iguales:  86.82211160597328 %


Dado que el porcentaje de valores distintos supone un 13% del total (mayor que el 5%), asumiremos que es un valor suficientemente importante como para tener en cuenta las StopWords y por tanto, las dejaremos para el estudio. 

- El vector de oraciones se llama: ```sentences```.
- El vector de palabras se llama: ```words```.

## Extracción de Características y Representación

Para la extracción de características vamos a usar dos métodos:
- **N-gramas de palabras**: usaremos 3-gramas, dado que se trata de una cantidad aceptable de elementos. 
- **Word Embedding**: usaremos word2vec para realizar la representación con una ponderación por la mediana para cada oración.


### 3-Gramas de Palabras

El vector de representación resultante se llama:  ```represent_ngramas```

In [120]:
print("Número de Oraciones", len(sentences))

Número de Oraciones 3817


In [121]:
ngram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(3,3))
data = ngram_vectorizer.fit_transform(sentences)
print(ngram_vectorizer.vocabulary_)

{'la solución es': 19112, 'solución es desarrollar': 37008, 'es desarrollar el': 12173, 'desarrollar el pensamiento': 8870, 'el pensamiento crítico': 10372, 'pensamiento crítico escéptico': 28004, 'hay que enseñar': 16356, 'que enseñar que': 31296, 'enseñar que la': 11802, 'que la magia': 31754, 'la magia no': 18632, 'magia no existe': 21946, 'que todos los': 32949, 'todos los avances': 39239, 'los avances de': 21011, 'avances de la': 2047, 'de la humanidad': 6854, 'la humanidad siempre': 18515, 'humanidad siempre han': 16829, 'siempre han sido': 36374, 'han sido pesar': 16135, 'sido pesar de': 36344, 'pesar de las': 28583, 'de las religiones': 6976, 'las religiones nunca': 19594, 'religiones nunca gracias': 34016, 'nunca gracias ellas': 26116, 'enseñar en las': 11800, 'en las escuelas': 11303, 'las escuelas la': 19406, 'escuelas la historia': 12810, 'la historia de': 18502, 'historia de las': 16674, 'las religiones todas': 19598, 'desde las religiones': 8921, 'las religiones de': 1958

In [124]:
represent_ngramas = data.toarray().astype(int)
print(represent_ngramas)
print("\nNúmero de Vectores de 3-gramas:", len(represent_ngramas))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Número de Vectores de 3-gramas: 3817


### Word Embedding con word2vec

El vector de representación resultante se llama:  ```rep_median_w_emb```

Dado que ya cargamos el corpus de Español de word2vec, no necesitamos cargarlo de nuevo. A continuación, procederemos a representar cada oración como un vector mediana de los word embdding de las palabras que lo forman. Usamos la **mediana** porque es una medida mucho más robusta que la media y puede servirnos para afinar mucho más. 

In [145]:
represent_w_embedding = []
y = len(wordvectors.get_vector(words[0][0]))
for orac in words:
  represent_frase = []
  for w in orac:
    try:
      represent_frase.append(wordvectors.get_vector(w))
    except KeyError:
      represent_frase.append(np.zeros(y))
  represent_w_embedding.append(represent_frase)

In [146]:
# Ejemplo de la primera oración de los word embedding 
# de cada palabra de cada oración.
represent_w_embedding[0]

[array([-7.1654e-02, -2.1225e-01,  9.8166e-02, -2.9260e-01,  7.0835e-02,
        -1.5002e-01, -1.3111e-01,  8.8604e-02, -2.0691e-01,  2.1911e-01,
        -8.0141e-02,  9.1307e-02,  5.7332e-02, -1.8983e-01,  1.1898e-01,
         2.0613e-02,  7.0634e-02,  1.1370e-02, -2.0079e-02, -9.5850e-02,
        -2.0350e-01,  1.6858e-01, -1.2561e-01, -9.2371e-02,  1.5820e-02,
        -2.1387e-02,  9.7338e-02,  9.3578e-02, -1.3777e-01,  1.1221e-01,
         2.7712e-02, -5.0668e-02,  3.6637e-02,  1.2425e-01, -1.0145e-01,
        -1.4884e-01, -2.9774e-03, -2.2499e-01,  6.9678e-02,  6.6174e-02,
        -5.6117e-02, -6.5538e-02, -4.7248e-02, -9.4947e-02,  1.8424e-01,
         2.6645e-01, -5.2417e-02,  5.4052e-02,  3.6522e-02,  5.1429e-02,
        -1.2625e-02,  3.4006e-02,  1.6534e-04, -7.1402e-02,  2.1150e-02,
        -2.0926e-01,  4.2031e-02, -1.6651e-01,  1.1762e-01,  8.4948e-02,
        -8.2924e-02, -1.2722e-01,  8.1386e-02,  1.1431e-01, -5.6041e-02,
         3.1826e-02,  1.0356e-01,  4.8049e-03, -8.6

In [147]:
rep_median_w_emb = [np.median(represent_w_embedding[i], axis=0) for i in range(len(represent_w_embedding))]

In [148]:
# Ejemplo de la primera oración de los word embedding 
# de cada palabra de cada oración.
rep_median_w_emb[0]

array([ 0.044679 , -0.11059  ,  0.0092639, -0.016607 , -0.025488 ,
        0.072145 , -0.097083 , -0.031994 , -0.20691  ,  0.069982 ,
       -0.18381  ,  0.027663 ,  0.11464  ,  0.035758 ,  0.11898  ,
        0.020613 ,  0.14825  ,  0.12717  , -0.020079 ,  0.090344 ,
       -0.2035   , -0.07237  , -0.092448 ,  0.049603 ,  0.041676 ,
       -0.13177  ,  0.14309  ,  0.093578 , -0.11674  ,  0.028425 ,
        0.036373 , -0.050668 ,  0.020796 ,  0.028661 , -0.034408 ,
       -0.14884  ,  0.05253  , -0.034474 ,  0.069678 ,  0.018879 ,
       -0.056117 , -0.057388 , -0.047248 , -0.094947 ,  0.01781  ,
        0.041773 , -0.034937 , -0.017204 ,  0.030079 ,  0.13787  ,
        0.099216 ,  0.024543 ,  0.11923  ,  0.07118  ,  0.02115  ,
       -0.15376  , -0.079235 , -0.17891  ,  0.060306 ,  0.14179  ,
       -0.0884   , -0.094113 ,  0.081386 ,  0.029121 , -0.012521 ,
        0.11658  ,  0.057616 ,  0.0048049, -0.0079941, -0.069213 ,
       -0.04697  ,  0.0090784,  0.026299 ,  0.050593 ,  0.0393